# Meshing introduction

gdsfactory has interfaces to external meshers (currently: [gmsh](https://gmsh.info/)).

Using a gdsfactory `Component` and a `Layerstack` reflecting the post-fabrication structure, you can automatically generate a 2D or 3D mesh suitable for physical simulation.

Within gdsfactory, this interface is currently used for:

* [finite-volume](https://en.wikipedia.org/wiki/Finite_volume_method) simulation through a [DEVSIM](https://devsim.org/) plugin, for e.g. TCAD simulations
* [finite-element](https://en.wikipedia.org/wiki/Finite_element_method) simulation through the [femwell](https://github.com/HelgeGehring/femwell) wrapper around [scikit-fem](https://scikit-fem.readthedocs.io/en/latest/), for e.g. mode solving and thermal simulations

Current features include:

* GDS postprocessing -- common interface for layout and simulation
* A generic shapely <--> gmsh translator, which properly reuses gmsh objects, resulting in conformal handling of
    * lateral interfaces
    * vertical interfaces
    * polygon inclusions
    * polygon "holes"
* 2D meshing of in-plane cross-sections (e.g. x - y)
* 2D meshing of out-of-plane cross-sections (e.g. arbitrary xy line - z)
* (In progress) 3D meshing
* The mesh is returned tagged with LayerStack `label` for each GDS layer according to a specific `mesh_order`
* All interfaces between layer entities are also tagged as `label1___label2` to e.g. implement interfacial boundary conditions
* Dummy layers can be easily introduced in a component to provide extra lines and polygons with custom labels to e.g. implement boundary conditions, sources, etc.
* Coarse resolution setting per label, and around interfaces
* Fine resolution setting with callable `[x,y,z,mesh_size]` functions (useful for simulation-driven refinement)

### Philosophy

GMSH can be used one of two ways:

* The traditional “bottom-up” way, where the user manually defines points, then line segments (from points), then lines (from segments), then closed curves (from lines), then surfaces (from curves), then closed shells (from surfaces), and finally volumes (from shells).
* With CAD-type boolean operations (set operations on objects)

While the latter method is much simpler for complex geometries, as of 2022 it does not preserve physical and mesh information, requiring manual "retagging" of the entities after the boolean operations, and driving its complexity back to bottom-up construction (especially for arbitrary geometries).

As such, gdsfactory uses the first approach, where the mask layers and a layerstack are used as a guide to define the various physical entities, which are returned as tagged objects to the user.

## Prerequisites

You can install the meshing plugins with `pip install gdsfactory[gmsh]`.

Note that PyVista may not work properly on headless systems.

## Usage

First, start with a gdsfactory `Component`

In [ ]:
import gdsfactory as gf

waveguide = gf.components.straight_pin(length=10, taper=None)
waveguide

and a `LayerStack`. Here, we copy the example from `gdsfactory.tech` for clarity). The `info` dict contains miscellaneous information about the layers, including `mesh_order`, which determines which layer will appear in the mesh if layers overlap.

In [ ]:
from gdsfactory.tech import LayerMap, LayerStack, LayerLevel

LAYER = LayerMap()
nm = 1e-3


def get_layer_stack_generic(
    thickness_wg: float = 220 * nm,
    thickness_slab_deep_etch: float = 90 * nm,
    thickness_clad: float = 3.0,
    thickness_nitride: float = 350 * nm,
    thickness_ge: float = 500 * nm,
    gap_silicon_to_nitride: float = 100 * nm,
    zmin_heater: float = 1.1,
    zmin_metal1: float = 1.1,
    thickness_metal1: float = 700 * nm,
    zmin_metal2: float = 2.3,
    thickness_metal2: float = 700 * nm,
    zmin_metal3: float = 3.2,
    thickness_metal3: float = 2000 * nm,
) -> LayerStack:
    """Returns generic LayerStack.

    based on paper https://www.degruyter.com/document/doi/10.1515/nanoph-2013-0034/html

    Args:
        thickness_wg: waveguide thickness in um.
        thickness_slab_deep_etch: for deep etched slab.
        thickness_clad: cladding thickness in um.
        thickness_nitride: nitride thickness in um.
        thickness_ge: germanium thickness.
        gap_silicon_to_nitride: distance from silicon to nitride in um.
        zmin_heater: TiN heater.
        zmin_metal1: metal1.
        thickness_metal1: metal1 thickness.
        zmin_metal2: metal2.
        thickness_metal2: metal2 thickness.
        zmin_metal3: metal3.
        thickness_metal3: metal3 thickness.
    """
    return LayerStack(
        layers=dict(
            core=LayerLevel(
                layer=LAYER.WG,
                thickness=thickness_wg,
                zmin=0.0,
                material="si",
                info={"mesh_order": 1},
            ),
            clad=LayerLevel(
                layer=LAYER.WGCLAD,
                zmin=0.0,
                material="sio2",
                thickness=thickness_clad,
                info={"mesh_order": 10},
            ),
            slab150=LayerLevel(
                layer=LAYER.SLAB150,
                thickness=150e-3,
                zmin=0,
                material="si",
                info={"mesh_order": 3},
            ),
            slab90=LayerLevel(
                layer=LAYER.SLAB90,
                thickness=thickness_slab_deep_etch,
                zmin=0.0,
                material="si",
                info={"mesh_order": 2},
            ),
            nitride=LayerLevel(
                layer=LAYER.WGN,
                thickness=thickness_nitride,
                zmin=thickness_wg + gap_silicon_to_nitride,
                material="sin",
                info={"mesh_order": 2},
            ),
            ge=LayerLevel(
                layer=LAYER.GE,
                thickness=thickness_ge,
                zmin=thickness_wg,
                material="ge",
                info={"mesh_order": 1},
            ),
            via_contact=LayerLevel(
                layer=LAYER.VIAC,
                thickness=zmin_metal1 - thickness_slab_deep_etch,
                zmin=thickness_slab_deep_etch,
                material="Aluminum",
                info={"mesh_order": 1},
            ),
            metal1=LayerLevel(
                layer=LAYER.M1,
                thickness=thickness_metal1,
                zmin=zmin_metal1,
                material="Aluminum",
                info={"mesh_order": 2},
            ),
            heater=LayerLevel(
                layer=LAYER.HEATER,
                thickness=750e-3,
                zmin=zmin_heater,
                material="TiN",
                info={"mesh_order": 1},
            ),
            via1=LayerLevel(
                layer=LAYER.VIA1,
                thickness=zmin_metal2 - (zmin_metal1 + thickness_metal1),
                zmin=zmin_metal1 + thickness_metal1,
                material="Aluminum",
                info={"mesh_order": 2},
            ),
            metal2=LayerLevel(
                layer=LAYER.M2,
                thickness=thickness_metal2,
                zmin=zmin_metal2,
                material="Aluminum",
                info={"mesh_order": 2},
            ),
            via2=LayerLevel(
                layer=LAYER.VIA2,
                thickness=zmin_metal3 - (zmin_metal2 + thickness_metal2),
                zmin=zmin_metal2 + thickness_metal2,
                material="Aluminum",
                info={"mesh_order": 1},
            ),
            metal3=LayerLevel(
                layer=LAYER.M3,
                thickness=thickness_metal3,
                zmin=zmin_metal3,
                material="Aluminum",
                info={"mesh_order": 2},
            ),
        )
    )

We can filter this stack to only focus on some layers:

In [ ]:
filtered_layerstack = LayerStack(
    layers={
        k: get_layer_stack_generic().layers[k]
        for k in (
            "slab90",
            "core",
            "via_contact",
        )
    }
)

In [ ]:
scene = waveguide.to_3d(layer_stack=filtered_layerstack)
scene.show()

The various processing and meshing functions are located under `gdsfactory.simulation.gmsh` and can be called from there, but a shortcut is implemented to mesh directly from a component:

In [ ]:
mesh = waveguide.to_gmsh(
    type="xy", z=0.09, layer_stack=filtered_layerstack, filename="mesh.msh"
)

This returns a gmsh `.msh` mesh, also saved in `filename` if provided, which can be processed:

In [ ]:
mesh.get_cells_type("triangle")

The `gmsh` GUI can be used to load and inspect the `.msh` file:

![msh mesh](https://imgur.com/jzwjEVC.png)

[meshio](https://github.com/nschloe/meshio) can also be used to convert the `.msh` to another arbitrary format, to observe for instance with `Paraview`. This is useful, for instance to preprocess the `msh` file using the `create_mesh` utility in order to consolidate entities with the same label:

In [ ]:
from gdsfactory.simulation.gmsh.mesh import create_physical_mesh
import meshio

mesh_from_file = meshio.read("mesh.msh")

triangle_mesh = create_physical_mesh(mesh_from_file, "triangle", prune_z=True)
meshio.write("mesh.xdmf", triangle_mesh)

Opening the `mesh.xdmf` in paraview:

![](https://imgur.com/zBn5596.png)

In [ ]:
line_mesh = create_physical_mesh(mesh_from_file, "line", prune_z=True)
meshio.write("facet_mesh.xdmf", line_mesh)

Opening the `facet_mesh.xdmf` in paraview:

![](https://imgur.com/tNhIIPK.png)

The `xdmf` files with consolidated physical groups can also be opened dynamically in a notebook with `pyvista`:

In [ ]:
import pyvista as pv

mesh = pv.read("mesh.xdmf")

plotter = pv.Plotter()
plotter.add_mesh(triangle_mesh, style="wireframe", show_scalar_bar=False)
plotter.show_grid()
plotter.camera_position = "xy"
plotter.show()